# Compute K-LL Auger rates for neon-like iron in a Debye-Hückel potential

In [1]:
using JAC

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b5211b19a5aa56f2c76a4f66df0ec7b35773f4c6-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b5211b19a5aa56f2c76a4f66df0ec7b35773f4c6-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

A plasma environment generally affects both, the level structure of atoms and ions as well their rates and cross sections. In this tutorial, we wish to compute the K-LL rates for neon-like iron in a Debye-Hückel potential following the work by Deprince *etal* (2019). 

A Debye-Hückel potential may affect the Auger rates of an atom or ion in three different ways: (i) the level structure due to the screened electron-nucleus and electron-electron interaction; (ii) the shape of the continuum potential due to the screened potential of the ionic core; as well as (iii) the screening of the electron-electron interaction in the Auger matrix. In the present implementation, only the (screened) Coulomb interaction can be taken into account for the computation of the plasma-modified Auger rates, i.e. **no** Breit interactions nor QED estimates are supported by the JAC program.

As with most other (atomic) processes, the compuation of plasma-modified Auger is supported in JAC by a special process identifier `AugerInPlasma` and the corresponding settings `PlasmaShift.AugerSettings`. These settings can be used to define the screening parameter $\lambda$. 


Let us start again from the `PlasmaShift.AugerSettings` to understand what and how we can control the computations:

In [2]:
? PlasmaShift.AugerSettings

`struct  PlasmaShift.AugerSettings`  ... defines a type for the details and parameters of computing Auger rates with plasma interactions.

```
+ plasmaModel            ::PlasmaModel                   ... Specified the particular type of plasma model; from {ion-sphere, debye}
+ lambdaDebye            ::Float64                       ... The lambda parameter of different plasma models.
+ ionSphereR0            ::Float64                       ... The effective radius of the ion-sphere model.
+ NoBoundElectrons       ::Int64                         ... Effective number of bound electrons.
+ printBeforeComputation ::Bool                          ... True, if all energies and lines are printed before their evaluation.
+ selectLines            ::Bool                          ... True, if lines are selected individually for the computations.
+ selectedLines          ::Array{Tuple{Int64,Int64},1}   ... List of lines, given by tupels (inital-level, final-level).
```

---

`JAC.PlasmaShift.AugerSettings()`  ... constructor for a standard instance of PlasmaShift.AugerSettings.


In [3]:
PlasmaShift.AugerSettings()   # show the default values

plasmaModel:             DebyeHueckel  
lambdaDebye:             0.25  
ionSphereR0:             0.0  
NoBoundElectrons:        0  
printBeforeComputation:  true  
selectLines:             false  
selectedLines:           Tuple{Int64,Int64}[]  


We can re-define these settings by specifying the screening parameter and the selected lines to be 
calculated:

In [4]:
pSettings = PlasmaShift.AugerSettings(PlasmaShift.DebyeHueckel, 0.1, 0., 0, true, true, [(1,1), (1,2)])

plasmaModel:             DebyeHueckel  
lambdaDebye:             0.1  
ionSphereR0:             0.0  
NoBoundElectrons:        0  
printBeforeComputation:  true  
selectLines:             true  
selectedLines:           Tuple{Int64,Int64}[(1, 1), (1, 2)]  



As usual, we can now defined and run the desired computation by *defining* and *performing* an 
`Atomic.Computation()`:


In [6]:
wa = Atomic.Computation("K-LL Auger spectrum in a Debye-Hueckel potential",  Nuclear.Model(26.); 
            grid = JAC.Radial.Grid("grid: by given parameters"; rnt = 2.0e-6, h = 5.0e-2, 
                                                                hp = 1.0e-2, NoPoints = 600), 
            initialConfigs=[Configuration("1s 2s^2 2p^6")],
            finalConfigs  =[Configuration("1s^2 2s^2 2p^4"), Configuration("1s^2 2s 2p^5"), 
                            Configuration("1s^2 2p^6")], process = JAC.AugerInPlasma,
            processSettings = pSettings)
perform(wa)

Define a radial grid of type MeshGL with 610 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=1.000e-02, NoPoints=600, r_max=3.229e+00;
  B-splines wit break points at every 7th point, nsL=86, nsS=87, orderL=7, orderS=8, orderGL=7] 
shellOccList = SubString{String}["1s", "2s^2", "2p^6"]
shellOccList = SubString{String}["1s^2", "2s^2", "2p^4"]
shellOccList = SubString{String}["1s^2", "2s", "2p^5"]
shellOccList = SubString{String}["1s^2", "2p^6"]

... in perform('computation: SCF', ...
perform-aa: Configuration: 1s_1/2^1 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.756573410087755 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]

  1s_1/2::  en [a.u.] = -2.89647168e+02;   self-consistency = 6.516769e-08  [1.796456e-07 for symmetry block kappa = -1]
  2s_1/2::  en [a.u.] = -5.32082804e+01;   self-consistency = 1.175601e-07  [1.796456e-07 for symmetry block kappa = -1]
  2p_1/2::  en [a.u.] = -4.98660048e+01;   self-consistency = 8.507489e-08  [1.089197e-07 for symmetry block kappa = 1]
  2p_3/2::  en [a.u.] = -4.93233162e+01;   self-consistency = 2.695664e-08  [3.405867e-08 for symmetry block kappa = -2]
Compute DebyeHueckel-CI matrix of dimension 1 x 1 for the symmetry 1/2^+ ...   ... done.

  Eigenenergies:

  Level  J Parity          Hartrees                    eV                   [eV]

     1    1/2 +     -8.455732871713233e+02    -2.300922112541919e+04    -2.300922112541919e+04 

  Energy of each level relative to immediately lower level:

  Level  J Parity          Hartrees                    eV                   [eV]


  Energy of each level relative to lowest level:

  Level  J Parity          Hartrees 

  1s_1/2::  en [a.u.] = -2.86150268e+02;   self-consistency = 4.997812e-05  [1.832106e-04 for symmetry block kappa = -1]
  2s_1/2::  en [a.u.] = -5.55652419e+01;   self-consistency = 7.889637e-05  [1.832106e-04 for symmetry block kappa = -1]
  2p_1/2::  en [a.u.] = -5.21894342e+01;   self-consistency = 6.161628e-05  [1.208746e-04 for symmetry block kappa = 1]
  2p_3/2::  en [a.u.] = -5.16738423e+01;   self-consistency = 2.417710e-05  [4.652023e-05 for symmetry block kappa = -2]

Iteration 5 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.86148413e+02;   self-consistency = 3.241216e-06  [1.181163e-05 for symmetry block kappa = -1]
  2s_1/2::  en [a.u.] = -5.55646779e+01;   self-consistency = 5.075442e-06  [1.181163e-05 for symmetry block kappa = -1]
  2p_1/2::  en [a.u.] = -5.21890504e+01;   self-consistency = 3.676928e-06  [7.211232e-06 for symmetry block kappa = 1]
  2p_3/2::  en [a.u.] = -5.16737090e+01;   self-consistency = 1.290190e-06  [2.479844e-06 for symmetry block kappa = -2]



┌ Warning: Adapt a proper continuum orbital for the plasma potential
└ @ JAC.Auger /home/fritzsch/fri/JAC.jl/src/module-Auger.jl:230


Continuum-Bessel orbital for q = 19.432951372691406,  l = 2
Pure-sine normalized continuum orbital with normalization constant N=3.9503e+00 (Delta-N=2.8611e+01) and phase phi=1.5085e+00 (Delta-N=8.4727e-01).
Continuum-Bessel orbital for q = 19.415693104561278,  l = 0
Pure-sine normalized continuum orbital with normalization constant N=2.6385e+00 (Delta-N=2.5201e+01) and phase phi=1.5638e+00 (Delta-N=8.4457e-01).


┌ Warning: Adapt a proper Auger amplitude for the plasma e-e interaction
└ @ JAC.Auger /home/fritzsch/fri/JAC.jl/src/module-Auger.jl:234
┌ Warning: Adapt a proper continuum orbital for the plasma potential
└ @ JAC.Auger /home/fritzsch/fri/JAC.jl/src/module-Auger.jl:230
┌ Warning: Adapt a proper Auger amplitude for the plasma e-e interaction
└ @ JAC.Auger /home/fritzsch/fri/JAC.jl/src/module-Auger.jl:234
┌ Warning: Adapt a proper continuum orbital for the plasma potential
└ @ JAC.Auger /home/fritzsch/fri/JAC.jl/src/module-Auger.jl:230
┌ Warning: Adapt a proper Auger amplitude for the plasma e-e interaction
└ @ JAC.Auger /home/fritzsch/fri/JAC.jl/src/module-Auger.jl:234


 
  Auger rates (without angular parameters): 

  -------------------------------------------------------------------------------------------------------------------
       i-level-f           i--J^P--f            Energy         Electron energy     Auger rate          alpha_2      
                                                 [eV]               [eV]              [1/s]                         
  -------------------------------------------------------------------------------------------------------------------
        1 --     1       1/2 + --> 2 +      -2.30092211e+04    5.13804845e+03    5.19511019e+17    0.00000000e+00    
        1 --     2       1/2 + --> 0 +      -2.30092211e+04    5.12892638e+03    2.59755509e+17    0.00000000e+00    
  -------------------------------------------------------------------------------------------------------------------
 
  Auger lifetimes, total rates and widths:
 
  -------------------------------------------------------------------------------


Similar computation will be made available also for photoionization cross sections in the near future.
